In [ ]:
from IPython.display import display, Markdown as md
display(md(f"# Social Media Monitoring"))
display(md(f"**Search for a Politician**"))
display(md(f"***The following table contains data on the politicians and their associated social acounts***"))

In [ ]:
import os
from smm_wrapper import SMM
import qgrid
from ipywidgets import widgets, Output
from IPython.display import display, clear_output
smm = SMM()

politicians_df = smm.dv.politicians_df()
qgrid_widget = qgrid.show_grid(politicians_df)
qgrid_widget

In [ ]:
display(md(f"**Select a politician**"))
def on_button_clicked(b):
    global politician
    
    # use the out widget so the output is overwritten when two or more
    # searches are performed
    with out:
        try:
            politician = smm.api.politician_search(politician_id=searchTerm.value)
            clear_output()
            display('Result found for #: {}. Name and last name: {} {}'.format(politician['politician_id'], 
                                                                               politician['firstname'], 
                                                                               politician['name']))

        except:
            clear_output()
            display(md(f'The politician with id *"{searchTerm.value}"* was not found. Please enter a number between 1 and 2516'))

# create and display the button
searchTerm = widgets.Text()
button = widgets.Button(description="Search")
example = md("Please enter a politician id from the table above. Example: *2193 for Angela Merkel*")
display(example, searchTerm, button)

# the output widge is used to remove the output after the search field
out = Output()
display(out)

# set the event
button.on_click(on_button_clicked)

# trigger the event with the default value
on_button_clicked(button)

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, curdoc
from bokeh.layouts import column
from bokeh.models.widgets import RadioButtonGroup
from bokeh.client import push_session
output_notebook()
import pandas as pd
from datetime import datetime

In [ ]:
def modify_agg(doc):
    #getting the data from SMM api
    def extracting_data(politician_id, agg):
        filtered_tweets = smm.api.tweets_by(politician_id=politician_id, aggregate_by=agg)
        filtered_replies = smm.api.reply_to(politician_id=politician_id, aggregate_by=agg)
        filtered_tweets['labels'] = [datetime.strptime(item, '%Y-%m-%d') for item in filtered_tweets['labels']]
        filtered_replies['labels'] = [datetime.strptime(item, '%Y-%m-%d') for item in filtered_replies['labels']]
        return(filtered_tweets, filtered_replies)
    
    #creating the plots
    filtered_tweets, filtered_replies = extracting_data(politician['politician_id'], agg='month')
    tw_y = filtered_tweets['values']
    tw_x = filtered_tweets['labels']
    re_y = filtered_replies['values']
    re_x = filtered_replies['labels']
    output_file("example.html")

    p = figure(title="Aggregated tweets and replies by Politician ID", x_axis_type='datetime', plot_height=400, plot_width=700,
                       x_range=[min(filtered_tweets['labels']), max(filtered_tweets['labels'])],
                       background_fill_color='#efefef', x_axis_label='Datetime', y_axis_label='Tweets')
    p.line(tw_x, tw_y, legend="number of tweets", line_width=2, line_color="blue")
    p.line(re_x, re_y, legend="number of replies", line_width=1, line_color="red")
    p.legend.location = "top_left"
    p.legend.click_policy="hide"
    def callback(attr, old, new):
        if new == 0:
            agg = 'day'
        elif new == 1:
            agg = 'week'
        else:
            agg = 'month'
        filtered_tweets, filtered_replies = extracting_data(politician['politician_id'], agg)
        
    #choosing the aggregation
    radio_button_group = RadioButtonGroup(
            labels=["Daily", "Weekly", "Monthly"], active=2)
    radio_button_group.on_change('active', callback)
    

    doc.add_root(column(radio_button_group, p))

In [ ]:
show(modify_agg, notebook_url="http://10.6.13.139:8020")